In [1]:
import numpy as np
from jitr import reactions, rmatrix
from jitr.reactions.potentials import woods_saxon_potential, coulomb_charged_sphere
from jitr.utils import kinematics, delta

define interaction

In [2]:
def interaction_local(r, V0, W0, R0, a0, Zz):
    nuclear = woods_saxon_potential(r, V0, W0, R0, a0)
    coulomb = coulomb_charged_sphere(r, Zz, R0)
    return nuclear + coulomb

In [3]:
def interaction_nonlocal(r, rp, V0, W0, R0, a0, Zz):
    return woods_saxon_potential(r, V0, W0, R0, a0) * np.exp(-((r - rp) ** 2))

In [4]:
def interaction_2level(r, V0, W0, V1, W1, g, R0, a0, Zz):
    nuclear0 = woods_saxon_potential(r, V0, W0, R0, a0)
    nuclear1 = woods_saxon_potential(r, V1, W1, R0, a0)
    coupling = woods_saxon_potential(r, g, 0, R0, a0)
    coulomb = coulomb_charged_sphere(r, Zz, R0)
    return np.array([[nuclear0 + coulomb, coupling], [coupling, nuclear1 + coulomb]])

In [5]:
def interaction_2level_nonlocal(r, rp, V0, W0, V1, W1, g, R0, a0, Zz):
    nuclear0 = woods_saxon_potential(r, V0, W0, R0, a0) * np.exp(-((r - rp) ** 2))
    nuclear1 = woods_saxon_potential(r, V1, W1, R0, a0) * np.exp(-((r - rp) ** 2))
    coupling = woods_saxon_potential(r, g, 0, R0, a0) * np.exp(-((r - rp) ** 2))
    return np.array([[nuclear0 + coulomb, coupling], [coupling, nuclear1 + coulomb]])

define system

In [6]:
Elab = 35  # MeV
Ca48 = (28, 20)
proton = (1, 1)

In [7]:
sys = reactions.ProjectileTargetSystem(
    channel_radii=np.array([5 * np.pi]),
    l=np.array([0]),
    mass_target=kinematics.mass(*Ca48),
    mass_projectile=kinematics.mass(*proton),
    Ztarget=Ca48[1],
    Zproj=proton[1],
)
Ecm, mu, k, eta = kinematics.classical_kinematics(
    sys.mass_target,
    sys.mass_projectile,
    Elab,
    sys.Ztarget * sys.Zproj,
)
channels, asymptotics = sys.build_channels(Ecm, mu, k, eta)

set up solver

In [8]:
solver = rmatrix.Solver(nbasis=40)

solve for a set of parameters

In [9]:
params = (42.0, 18.1, 4.8, 0.7, sys.Zproj * sys.Ztarget)
R, S, uext_boundary = solver.solve(
    channels, asymptotics, interaction_local, args_local=params
)

/home/beyerk/Projects/jitr/src/jitr/rmatrix/rmatrix_solver.py:44: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (Array(complex128, 1, 'C', False, aligned=True), Array(complex128, 2, 'A', False, aligned=True))
  R, Ainv = rmatrix_with_inverse(A, b, nchannels, nbasis, a)


get phase shift in degrees

In [10]:
phase_shift, phase_attenuation = delta(S[0, 0])
print(f"phase shift: {phase_shift:1.3f} + i ({phase_attenuation:1.3f}) [degrees]")

phase shift: -70.751 + i (-3.439) [degrees]


In [11]:
V = np.diag(solver.kernel.matrix_local(interaction_local, 5 * np.pi, params))
V.reshape(1, 1, 40, 40).swapaxes(1, 2).reshape(40, 40) - V

array([[0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       ...,
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]])